In [65]:
"""
Preamble for most code and jupyter notebooks
@author: bridgetsmart
@notebook date: 6 Jun 2022
"""

import numpy as np, pandas as pd

import matplotlib.pyplot as plt, seaborn as sns
import matplotlib as mpl

import math, string, re, pickle, json, time, os, sys, datetime, itertools
from ProcessEntropy.SelfEntropy import self_entropy_rate

from tqdm.notebook import tqdm

# Set panda's options
pd.set_option("display.max_rows", 40)
pd.set_option("display.max_columns", 120)

# Graphics for thesis
%config InlineBackend.figure_formats = ['retina']
params = {
    'axes.labelsize': 10,
    'font.size': 10,
    'legend.fontsize': 8,
    'xtick.labelsize': 8,
    'ytick.labelsize': 8,
    'figure.figsize': [5.3, 3.34], # Thesis width
    'figure.dpi' : 72, 
    'font.family': "serif",
    'text.usetex': False, # Use LaTeX when desired
    }
plt.rcParams.update(params)

# Set packages to autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Fit regular Markov Chain

In [108]:
def fit_to_markov(data):
    # need to map all to a 0,n-1 alphabet
    # then use the markov chain to get the probabilities

    # number of states
    states = [y for x in data for y in x]

    # first, get the alphabet
    alphabet = list(set(states))
    alphabet.sort()
    n = len(alphabet)
    # now, map the data to the alphabet
    data_mapped = np.array([[alphabet.index(y) for y in x] for x in data])
    states = [alphabet.index(x) for x in states]
    # get all one steps    
    one_step_trans = [(x[i], x[i+1]) for x in data_mapped for i in range(len(x)-1)]

    # transition matrix
    A = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            A[i, j] = one_step_trans.count((i, j))

    # # initial distribution
    # pi = np.zeros(n)
    # for i in range(n):
    #     pi[i] = states.count(i)

    
    # normalize
    A /= A.sum(axis=1)[:, None]

    return A, n


### get entropy from transition matrix

In [45]:
def get_ent(pi,A,n):
    # get the entropy
    ent = 0
    for i in range(n):
        for j in range(n):
            if A[i,j] ==0:
                pass
            else:
                ent += - pi[i]*A[i,j]*math.log(A[i,j])
    return ent    

In [107]:
# function to get entropy estimates from LAMP fit
def get_stationary(A):
    e_vals, eig_vecs = np.linalg.eig(A.T) # gets left e vectors
    l = np.where(abs(e_vals-1) == np.min(abs(e_vals-1)))[0][0]
    # get real component
    v = eig_vecs[:,l].real
    return v/sum(v)


### get shannon entropy + return all

In [ ]:
def ent_est(data):
    s = [y for x in data for y in x]
    #### do once by appending all
    print('getting estimate 1...')
    est1 = self_entropy_rate(s)

    #### do by averaging each path
    print('getting estimate 2...')
    est2 = np.mean([self_entropy_rate(x) for x in tqdm(data)])

    # fit markov
    print('getting estimate 3...')
    A,n = fit_to_markov(data)
    pi = get_stationary(A)
    est3 = get_ent(pi,A,n)

    #### shannon estimator
    print('getting estimate 4...')
    est4 = np.sum(-pi*np.log(pi))

    return est1, est2, est3, est4

# read in transition matrix

In [ ]:
data_path = ... 

In [153]:
def get_trans(fn):
    # read in transition matrix
    with open(data_path+fn, 'r') as f:
        lamp_transitions = np.array([y.replace("\n","").split(',') for y in f.readlines()])
        n_lamp = np.max(list(set([int(x) for x in lamp_transitions[:,2]] + [int(x) for x in lamp_transitions[:,1]])))+1
        lamp_A = np.zeros((n_lamp,n_lamp))
        for r in lamp_transitions:
            lamp_A[int(r[1]),int(r[2])] = float(r[0]) # value
        return lamp_A

## Wikispedia

In [155]:
lamp_A = get_trans("wiki_transition_matrix.txt")

In [125]:
# read in data
with open(data_path + 'wiki_paths.txt', 'r') as f:
    data = [x.replace("\n","").split(";") for x in f.readlines()]
 

In [158]:
# get all estimates

e1,e2,e3,e4 = ent_est(data)

## Reuters

## Brightkite

## last-fm